<a href="https://colab.research.google.com/github/sainikhila11/INFO5731_Group2_FinalProject/blob/main/MedMentions_Dataset_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 2.3 MB/s eta 0:00:00


In [ ]:
import re

In [ ]:
import pandas as pd

# read csv
output_df = pd.read_csv("output (1).csv")

# Read csv
snomed_df = pd.read_csv("protien.csv")

# Define a function to find matching content based on SemanticTypeID
def find_matching_content(SemanticID):
    # Check if the SemanticTypeID exists in snomed_df
    if SemanticID in snomed_df['SemanticTypeID'].astype(str).values:
        # Find the corresponding SemanticTypeName content
        matching_content = snomed_df.loc[snomed_df['SemanticTypeID'].astype(str) == SemanticID, 'SemanticTypeName'].values[0]
        return matching_content
    else:
        return None

# Add a new column containing the matching content to output_df
output_df['MatchingContent'] = output_df['SemanticTypeID'].apply(find_matching_content)

# Write the updated dataframe back to output.csv
output_df.to_csv("output_with_matching_content.csv", index=False)


In [ ]:
output_df

,PMID,StartIndex,EndIndex,MentionTextSegment,SemanticTypeID,EntityID,MatchingContent
0,25763772,0.0,5.0,DCTN4,T116,C4308010,"Amino Acid, Peptide, or Protein"
1,25763772,0.0,5.0,DCTN4,T123,C4308010,Biologically Active Substance
2,25763772,23.0,63.0,chronic Pseudomonas aeruginosa infection,T047,C0854135,Disease or Syndrome
3,25763772,67.0,82.0,cystic fibrosis,T047,C0010674,Disease or Syndrome
4,25763772,83.0,120.0,Pseudomonas aeruginosa (Pa) infection,T047,C0854135,Disease or Syndrome
...,...,...,...,...,...,...,...
382315,28550521,1638.0,1651.0,Bone sounding,T060,C0430022,Diagnostic Procedure
382316,28550521,1660.0,1667.0,highest,T080,C1522410,Qualitative Concept
382317,28550521,1668.0,1676.0,accuracy,T080,C0443131,Qualitative Concept
382318,28550521,1690.0,1701.0,interdental,T082,C0442104,Spatial Concept


In [ ]:
import openai
import pandas as pd
import re

# Define your OpenAI API key
api_key = 'sk-proj-9gpI8AenRrdW9leB5KM6T3BlbkFJms1AEo0USSfymPGzCZmt'
openai.api_key = api_key

# Function to read the dataset in chunks
def read_dataset(file_path, chunk_size):
    chunks = pd.read_csv(file_path, chunksize=chunk_size)
    return chunks

# Function to generate prompt based on dataset information
def generate_prompt(dataset_chunk):
    prompt = f"dont print anything other than percentage of the comprehensiveness of the provided biomedical dataset \n{dataset_chunk}\n by analyzing the alignment between the MentionTextSegment and MatchingContent columns. The MentionTextSegment column contains mentions of entities or terms, while the MatchingContent column provides the corresponding semantic classification or content associated with each mention.\nAssess the extent to which the MentionTextSegment values accurately match the semantic classifications or content provided in the MatchingContent column. Consider each pair of mention text and matching content as a unit of analysis.\nTo determine comprehensiveness, count the number of correctly matched pairs and calculate the percentage of correct matches relative to the total number of pairs in the dataset. Higher percentages indicate a more comprehensive alignment between mention text and matching content."
    return prompt

# Main function to assess dataset comprehensiveness
def assess_dataset_comprehensiveness(file_path, chunk_size=10, total_rows_limit=1000):
    # Read the dataset in chunks
    dataset_chunks = read_dataset(file_path, chunk_size)

    # Initialize variables to store total rows and duplicated combinations
    total_rows = 0
    duplicated_combinations = 0

    # Process each chunk
    for chunk in dataset_chunks:
        # Increment total rows count
        total_rows += len(chunk)

        # Check if total rows limit reached
        if total_rows > total_rows_limit:
            break

        # Generate prompt based on dataset chunk
        prompt = generate_prompt(chunk)

        # Request assessment from OpenAI
        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=prompt,
            temperature=0.5,
            max_tokens=400,
            n=1,
            stop=None,
            timeout=None
        )

        # Get the assessment result from the response
        percentage_duplicated_str = response.choices[0].text.strip()

        # Extract the percentage value
        percentage_duplicated_match = re.search(r'\d+', percentage_duplicated_str)
        if percentage_duplicated_match:
            percentage_duplicated = int(percentage_duplicated_match.group())
            # Update duplicated combinations count
            duplicated_combinations += percentage_duplicated

    # Calculate percentage of duplicated combinations
    percentage_duplicated_total = (duplicated_combinations / (2*total_rows_limit)) * 100

    return percentage_duplicated_total

# Example usage
file_path = "output_with_matching_content.csv"
percentage_duplicated = assess_dataset_comprehensiveness(file_path, chunk_size=10, total_rows_limit=1000)
print("Percentage of comprehensiveness of combinations:", percentage_duplicated)


Percentage of comprehensiveness of combinations: 62.64999999999999


In [ ]:
import openai
import pandas as pd
import re

# Define your OpenAI API key
api_key = 'sk-proj-9gpI8AenRrdW9leB5KM6T3BlbkFJms1AEo0USSfymPGzCZmt'
openai.api_key = api_key

# Function to read the dataset in chunks
def read_dataset(file_path, chunk_size):
    chunks = pd.read_csv(file_path, chunksize=chunk_size)
    return chunks

# Function to generate prompt based on dataset information
def generate_prompt(dataset_chunk):
    prompt = f"dont print anything other than percentage of the correctness of the provided biomedical dataset \n{dataset_chunk}\n calculate by comparing the MentionTextSegment and MatchingContent columns. The MentionTextSegment column contains mentions of entities or terms, while the MatchingContent column provides the corresponding semantic classifications or content associated with each mention.\nExamine each pair of mention text and matching content to determine if they align correctly. output only percentage"
    return prompt

# Main function to assess dataset correctness
def assess_dataset_correctness(file_path, chunk_size=10, total_rows_limit=1000):
    # Read the dataset in chunks
    dataset_chunks = read_dataset(file_path, chunk_size)

    # Initialize variables to store total rows and correct combinations
    total_rows = 0
    correct_combinations = 0

    # Process each chunk
    for chunk in dataset_chunks:
        # Increment total rows count
        total_rows += len(chunk)

        # Check if total rows limit reached
        if total_rows > total_rows_limit:
            break

        # Generate prompt based on dataset chunk
        prompt = generate_prompt(chunk)

        # Request assessment from OpenAI
        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=prompt,
            temperature=0.5,
            max_tokens=400,
            n=1,
            stop=None,
            timeout=None
        )

        # Get the assessment result from the response
        correctness_str = response.choices[0].text.strip()

        # Extract the correctness percentage
        correctness_match = re.search(r'\d+', correctness_str)
        if correctness_match:
            correctness = int(correctness_match.group())
            # Update correct combinations count
            correct_combinations += correctness

    # Calculate percentage of correctness
    percentage_correctness = (correct_combinations / (total_rows_limit))*chunk_size

    return percentage_correctness

# Example usage
file_path = "output_with_matching_content.csv"
percentage_correctness = assess_dataset_correctness(file_path, chunk_size=10, total_rows_limit=1000)
print("Percentage of correctness of combinations:", percentage_correctness)


Percentage of correctness of combinations: 79.09


In [ ]:
import openai
import pandas as pd
import re

# Define your OpenAI API key
api_key = 'sk-proj-9gpI8AenRrdW9leB5KM6T3BlbkFJms1AEo0USSfymPGzCZmt'
openai.api_key = api_key

# Function to read the dataset in chunks
def read_dataset(file_path, chunk_size):
    chunks = pd.read_csv(file_path, chunksize=chunk_size)
    return chunks

# Function to generate prompt based on dataset information
def generate_prompt(dataset_chunk):
    prompt = f"dont print anything other than percentage of  variety of dataset consider the unique combinations between the 'MentionTextSegment' and 'MatchingContent' columns in the provided biomedical dataset \n{dataset_chunk}\n. check the unique combinations of values and calculate their percentage relative to the total number of rows in the dataset. Assess the diversity and coherence of the dataset by considering the alignment between the terms mentioned in the 'MentionTextSegment' column and their corresponding semantic classifications in the 'MatchingContent' column."
    return prompt

# Main function to assess dataset variety
def assess_dataset_comprehensiveness(file_path, chunk_size=10, total_rows_limit=1000):
    # Read the dataset in chunks
    dataset_chunks = read_dataset(file_path, chunk_size)

    # Initialize variables to store total rows and duplicated combinations
    total_rows = 0
    duplicated_combinations = 0

    # Process each chunk
    for chunk in dataset_chunks:
        # Increment total rows count
        total_rows += len(chunk)

        # Check if total rows limit reached
        if total_rows > total_rows_limit:
            break

        # Generate prompt based on dataset chunk
        prompt = generate_prompt(chunk)

        # Request assessment from OpenAI
        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=prompt,
            temperature=0.5,
            max_tokens=400,
            n=1,
            stop=None,
            timeout=None
        )

        # Get the assessment result from the response
        percentage_duplicated_str = response.choices[0].text.strip()

        # Extract the percentage value
        percentage_duplicated_match = re.search(r'\d+', percentage_duplicated_str)
        if percentage_duplicated_match:
            percentage_duplicated = int(percentage_duplicated_match.group())
            # Update variety combinations count
            duplicated_combinations += percentage_duplicated

    # Calculate percentage of variety combinations
    percentage_duplicated_total = (duplicated_combinations / (2*total_rows_limit))*100

    return percentage_duplicated_total

# Example usage
file_path = "output_with_matching_content.csv"
percentage_duplicated = assess_dataset_comprehensiveness(file_path, chunk_size=10, total_rows_limit=1000)
print("Percentage of variety of combinations:", percentage_duplicated)


Percentage of variety of combinations: 55.800000000000004


In [ ]:
import openai
import pandas as pd
import re

# Define your OpenAI API key
api_key = 'sk-proj-9gpI8AenRrdW9leB5KM6T3BlbkFJms1AEo0USSfymPGzCZmt'
openai.api_key = api_key

# Function to read the dataset in chunks
def read_dataset(file_path, chunk_size):
    chunks = pd.read_csv(file_path, chunksize=chunk_size)
    return chunks

# Function to generate prompt based on dataset information
def generate_prompt(dataset_chunk):
    prompt = f"dont print anything other than percentage of duplicated , combinations between the 'MentionTextSegment' and 'MatchingContent' columns in the provided biomedical dataset \n{dataset_chunk}\n. check the combinations of values that occur more than once and determine their percentage relative to the total number of rows in the dataset.\n This metric serves as a measure of duplication within the dataset, reflecting the redundancy or repetition of certain combinations.\n Return the percentage of duplicated combinations as the output."
    return prompt

# Main function to assess dataset duplicated
def assess_dataset_comprehensiveness(file_path, chunk_size=10, total_rows_limit=1000):
    # Read the dataset in chunks
    dataset_chunks = read_dataset(file_path, chunk_size)

    # Initialize variables to store total rows and duplicated combinations
    total_rows = 0
    duplicated_combinations = 0

    # Process each chunk
    for chunk in dataset_chunks:
        # Increment total rows count
        total_rows += len(chunk)

        # Check if total rows limit reached
        if total_rows > total_rows_limit:
            break

        # Generate prompt based on dataset chunk
        prompt = generate_prompt(chunk)

        # Request assessment from OpenAI
        response = openai.Completion.create(
            engine="gpt-3.5-turbo-instruct",
            prompt=prompt,
            temperature=0.5,
            max_tokens=400,
            n=1,
            stop=None,
            timeout=None
        )

        # Get the assessment result from the response
        percentage_duplicated_str = response.choices[0].text.strip()

        # Extract the percentage value
        percentage_duplicated_match = re.search(r'\d+', percentage_duplicated_str)
        if percentage_duplicated_match:
            percentage_duplicated = int(percentage_duplicated_match.group())
            # Update duplicated combinations count
            duplicated_combinations += percentage_duplicated

    # Calculate percentage of duplicated combinations
    percentage_duplicated_total = (duplicated_combinations / (2*total_rows_limit)) * 100
    return percentage_duplicated_total

# Example usage
file_path = "output_with_matching_content.csv"
percentage_duplicated = assess_dataset_comprehensiveness(file_path, chunk_size=10, total_rows_limit=1000)
print("Percentage of duplicated combinations:", percentage_duplicated)


Percentage of duplicated combinations: 63.949999999999996
